In [ ]:
#@title Installations
# Basic stuff
! pip install langchain langchain_community langchain_chroma
! pip install -qU langchain-openai

# For loading PDF's
! pip install --upgrade --quiet pypdf

# For indexing PDF's
! pip install --upgrade --quiet faiss-cpu
# use `pip install faiss-gpu` for CUDA GPU support

# For sentence embedding with HuggingFace
! pip install sentence-transformers

# AutoModelForCausalLM and AutoTokenizer are used to load the model and tokenizer.
! pip install transformers torch

# For the open-source llm
! pip install langchain_huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.9/394.9 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.1/149.1 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.9 MB/s et

In [ ]:
#@title Setting OpenAI's and LangChain's API keys
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()



··········
··········


In [ ]:
#@title Load the LLM -- OpenAI

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")


In [ ]:
#@title Load the LLM -- Open-Source

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain import HuggingFacePipeline

# Load a model and tokenizer that supports Spanish and is suited for text generation
# model_name = "EleutherAI/gpt-j-6B"  # too heavy
model_name = "EleutherAI/gpt-neo-2.7B"

model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Create a Hugging Face pipeline for text generation
hf_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Wrap the Hugging Face pipeline in a LangChain-compatible LLM class
llm = HuggingFacePipeline(pipeline=hf_pipeline)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/10.7G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:151: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


## 1. Indexing: Load


In [ ]:
# Upload a PDF
from google.colab import files
uploaded_file = files.upload()


Saving codigo_penal.pdf to codigo_penal.pdf


In [ ]:
from langchain_community.document_loaders import PyPDFLoader

file_path = (
    "./codigo_penal.pdf"
)
loader = PyPDFLoader(file_path)
pages = loader.load_and_split()

pages[0]

Document(metadata={'source': './codigo_penal.pdf', 'page': 0}, page_content='Código Penal\nLey 11.179\nBUENOS AIRES, 21 de Diciembre de 1984\nBoletín Oficial, 16 de Enero de 1985\nVigente, de alcance general\nEnciclopedia: Integridad y Transparencia\nId SAIJ: LNS0002677\nTEMA\nCódigo penal, modificación del Código Penal, integridad en el ejercicio de la función pública, delitos cometidos\npor funcionarios públicos, incumplimiento de los deberes del funcionario público, enriquecimiento injustificado en\nla función pública, funcionarios públicos, empleados públicos, deberes del empleado público, abuso de\nautoridad, inhabilidades especiales de los funcionarios públicos, personal de las fuerzas de seguridad, delitos\ncontra la integridad\nCODIGO PENAL DE LA NACION ARGENTINA L I B R O P R I M E R O D I S P O S I C I\nO N E S G E N E R A L E S\nTITULO I APLICACION DE LA LEY PENAL\nARTICULO 1. - Este Código se aplicará:\n1) Por delitos cometidos o cuyos efectos deban producirse en el territo

In [ ]:
print(len(pages[0].page_content))
print(pages[0].page_content)
print(pages[0].metadata)

1886
Código Penal
Ley 11.179
BUENOS AIRES, 21 de Diciembre de 1984
Boletín Oficial, 16 de Enero de 1985
Vigente, de alcance general
Enciclopedia: Integridad y Transparencia
Id SAIJ: LNS0002677
TEMA
Código penal, modificación del Código Penal, integridad en el ejercicio de la función pública, delitos cometidos
por funcionarios públicos, incumplimiento de los deberes del funcionario público, enriquecimiento injustificado en
la función pública, funcionarios públicos, empleados públicos, deberes del empleado público, abuso de
autoridad, inhabilidades especiales de los funcionarios públicos, personal de las fuerzas de seguridad, delitos
contra la integridad
CODIGO PENAL DE LA NACION ARGENTINA L I B R O P R I M E R O D I S P O S I C I
O N E S G E N E R A L E S
TITULO I APLICACION DE LA LEY PENAL
ARTICULO 1. - Este Código se aplicará:
1) Por delitos cometidos o cuyos efectos deban producirse en el territorio de la Nación Argentina, o en los
lugares sometidos a su jurisdicción.
2) Por delitos 

In [ ]:
import pandas as pd
lenghts = [len(pages[i].page_content) for i in range(len(pages))]
length = pd.Series(lenghts, name='chunk_size')
length.describe()

,chunk_size
count,78.000000
mean,2709.346154
std,507.317847
min,372.000000
25%,2480.750000
50%,2679.000000
75%,3024.750000
max,3883.000000


## 2. Indexing: Split

In [ ]:
#@title OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

all_splits = text_splitter.split_documents(pages)

all_splits[0]

Document(metadata={'source': './codigo_penal.pdf', 'page': 0}, page_content='Código Penal\nLey 11.179\nBUENOS AIRES, 21 de Diciembre de 1984\nBoletín Oficial, 16 de Enero de 1985\nVigente, de alcance general\nEnciclopedia: Integridad y Transparencia\nId SAIJ: LNS0002677\nTEMA\nCódigo penal, modificación del Código Penal, integridad en el ejercicio de la función pública, delitos cometidos\npor funcionarios públicos, incumplimiento de los deberes del funcionario público, enriquecimiento injustificado en\nla función pública, funcionarios públicos, empleados públicos, deberes del empleado público, abuso de\nautoridad, inhabilidades especiales de los funcionarios públicos, personal de las fuerzas de seguridad, delitos\ncontra la integridad\nCODIGO PENAL DE LA NACION ARGENTINA L I B R O P R I M E R O D I S P O S I C I\nO N E S G E N E R A L E S\nTITULO I APLICACION DE LA LEY PENAL\nARTICULO 1. - Este Código se aplicará:\n1) Por delitos cometidos o cuyos efectos deban producirse en el territo

In [ ]:
#@title For the Open-Source model
from langchain.text_splitter import RecursiveCharacterTextSplitter

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=50)
# achico esto porque the GPT-Neo-2.7B model can handle up to 2048 tokens as input

all_splits = text_splitter.split_documents(pages)

all_splits[0]

Document(metadata={'source': './codigo_penal.pdf', 'page': 0}, page_content='Código Penal\nLey 11.179\nBUENOS AIRES, 21 de Diciembre de 1984\nBoletín Oficial, 16 de Enero de 1985')

## 3. Indexing: Store

In [ ]:
#@title OpenAI
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

In [ ]:
#@title Open-Source
from langchain_chroma import Chroma
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings

# Load a pre-trained model from SentenceTransformers
# model_name = "all-MiniLM-L6-v2"
model_name = "distiluse-base-multilingual-cased-v2"
# sentence_transformer = SentenceTransformer(model_name)

# Use HuggingFaceEmbeddings with the SentenceTransformers model
# embedding = HuggingFaceEmbeddings(model=sentence_transformer)
embedding = HuggingFaceEmbeddings(model_name=model_name)

# Create the vector store with the documents and the open-source embeddings
vectorstore = Chroma.from_documents(documents=all_splits, embedding=embedding)


KeyboardInterrupt: 

## 4. Retrieval and Generation: Retrieve


In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

question = "¿Hay artículos que prevean delitos cometidos por argentinos en el extrangero?"

retrieved_docs = retriever.invoke(question)

len(retrieved_docs)

6

In [ ]:
retrieved_docs[0]

Document(metadata={'page': 0, 'source': './codigo_penal.pdf'}, page_content='ARTICULO 1. - Este Código se aplicará:\n1) Por delitos cometidos o cuyos efectos deban producirse en el territorio de la Nación Argentina, o en los\nlugares sometidos a su jurisdicción.\n2) Por delitos cometidos en el extranjero por agentes o empleados de autoridades argentinas en desempeño de\nsu cargo.\n3) Por el delito previsto en el artículo 258 bis cometido en el extranjero, por ciudadanos argentinos o personas\njurídicas con domicilio en la República Argentina, ya sea aquel fijado en sus estatutos o el correspondiente a los\nestablecimientos o sucursales que posea en el territorio argentino.\nARTICULO 2. - Si la ley vigente al tiempo de cometerse el delito fuere distinta de la que exista al pronunciarse\nel fallo o en el tiempo intermedio, se aplicará siempre la más benigna.\nSi durante la condena se dictare una ley más benigna, la pena se limitará a la establecida por esa ley.\nEn todos los casos del pr

## 5. Retrieval and Generation: Generate

In [ ]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

example_messages

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filler question \nContext: filler context \nAnswer:")]

Tengo que reemplazar llm por algo open-source.

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

for chunk in rag_chain.stream("What is Task Decomposition?"):
    print(chunk, end="", flush=True)

I don't know.

In [ ]:
question = "¿Hay artículos que prevean delitos cometidos por argentinos en el extrangero?"
rag_chain.invoke(question)

'Sí, hay artículos que prevén delitos cometidos por argentinos en el extranjero. En particular, el artículo 1 establece que se aplicará la ley argentina a delitos cometidos en el extranjero por ciudadanos argentinos o personas jurídicas con domicilio en Argentina. Además, el artículo 258 bis menciona específicamente delitos cometidos en el extranjero por ciudadanos argentinos.'